# Factor Selection

In [27]:
from WindPy import w
import pandas as pd
import numpy as np
import os
import shutil
import statsmodels.api as sm
import matplotlib.pyplot as plt
from statsmodels.stats.diagnostic import acorr_ljungbox
from statsmodels.stats.api import het_white, het_breuschpagan
from statsmodels.stats.sandwich_covariance import cov_hac
from statsmodels.tsa.seasonal import STL

In [ ]:
w.start()

In [ ]:
start_date = "2008-12-01" # actually from 2010, extract data for preprocessing
end_date = "2024-05-31"

In [ ]:
# takes a dict of indicators and their corresponding ticker in Wind to
# create a DataFrame containing the data extracted from Wind API
def extract_inds(inds):
    df = pd.DataFrame()
    for ind in inds:
        raw = w.edb(inds[ind], start_date, end_date)
        # Extract data from the Wind API response
        times = raw.Times
        data = raw.Data[0]
        # Convert times to pandas datetime format
        times = pd.to_datetime([str(time) for time in times], format="%Y-%m-%d")
        
        # Create a temporary DataFrame for the current indicator
        temp_df = pd.DataFrame(data, index=times, columns=[ind])
        
        # Combine the temporary DataFrame with the main DataFrame
        if df.empty:
            df = temp_df
        else:
            df = df.join(temp_df, how='outer')
    return df

In [ ]:
def lag_data(df, periods=1):
    df_lagged = df.shift(periods)
    df_lagged = df_lagged.iloc[periods:]  # Remove the first rows
    return df_lagged

In [28]:
def apply_x11(df, columns, x13as_path='/usr/local/bin/x13as'):
    tempdir = os.path.join(os.getcwd(), 'x13as_temp')
    os.makedirs(tempdir, exist_ok=True)

    df_adjusted = df.copy()
    for column in columns:
        ts = df[column].asfreq('M')
        
        # Handle zero or negative values
        min_value = ts.min()
        shift_value = abs(min_value) + 1 if min_value <= 0 else 0
        ts_shifted = ts + shift_value
        
        try:
            result = sm.tsa.x13_arima_analysis(ts_shifted, x12path=x13as_path, tempdir=tempdir)
            if result is None:
                raise ValueError(f"X-13ARIMA-SEATS returned None for column: {column}")
            
            # Adjust the seasonally adjusted data back
            df_adjusted[column] = result.seasadj - shift_value
            
        except Exception as e:
            print(f"Error processing column {column}: {e}")
            
            # Read and print the .err file for more details
            err_file_path = os.path.join(tempdir, 'x13as.err')
            if os.path.exists(err_file_path):
                with open(err_file_path, 'r') as err_file:
                    print(f"Error file content for column {column}:")
                    print(err_file.read())
            continue
    
    # Cleanup temporary directory
    shutil.rmtree(tempdir)

    return df_adjusted

In [ ]:
# seasonal adjustment
def apply_stl(df, columns, seasonal=4):
    df_adjusted = df.copy()
    for column in columns:
        stl = STL(df[column], seasonal=seasonal)
        result = stl.fit()
        df_adjusted[column] = result.seasonal  # Replace with seasonally adjusted component
    return df_adjusted

In [ ]:
# Logarithmic YoY Change Processing
def logyoy(df, columns):
    df_log_yoy = df.copy()
    for column in columns:
        df_log_yoy[column] = np.log(df[column] / df[column].shift(12)) * 100
    df_log_yoy = df_log_yoy.iloc[12:]  # Remove the first 12 rows where the shift operation would result in NaN
    return df_log_yoy

In [ ]:
# for factor selection, regress on the exchang rate to observe Beta, T-Value, and R-Squared
# and record the median values after 1000 iterations
def regress(indicator_df, forex_df, target_column='USD/CNY Central Parity Rate', n_iterations=1000, min_period_length=24):
    # Initialize a DataFrame to store regression results
    forex_df.index = indicator_df.index
    results = []

    for i in range(n_iterations):
        # Resample with replacement
        sample_indices = np.random.choice(indicator_df.index, size=len(indicator_df), replace=True)
        sample_growth_df = indicator_df.loc[sample_indices]
        sample_forex_df = forex_df.loc[sample_indices]
        
        # Randomly select start and end dates
        start_idx = np.random.randint(0, len(sample_growth_df) - min_period_length)
        end_idx = start_idx + min_period_length
        
        # Subset the data to ensure period longer than 2 years
        sample_growth_period = sample_growth_df.iloc[start_idx:end_idx]
        sample_forex_period = sample_forex_df.iloc[start_idx:end_idx]
        
        # Perform regression for each indicator
        for indicator in indicator_df.columns:
            X = sample_growth_period[[indicator]]
            X = sm.add_constant(X)
            y = sample_forex_period[target_column]
            
            model = sm.OLS(y, X).fit(cov_type='HAC', cov_kwds={'maxlags':1})
            beta = model.params[indicator]
            t_value = model.tvalues[indicator]
            r_squared = model.rsquared
            
            # Append the results
            results.append({
                'Indicator': indicator,
                'Beta': beta,
                'T-Value': t_value,
                'R-Squared(%)': r_squared * 100  # Convert to percentage
            })
            
    # Convert results to DataFrame
    growth_results_df = pd.DataFrame(results)

    # Compute median values
    growth_median_results = growth_results_df.groupby('Indicator').median().reset_index()

    # Format the median results
    growth_median_results['Beta'] = growth_median_results['Beta'].map("{:.2f}".format)
    growth_median_results['T-Value'] = growth_median_results['T-Value'].map("{:.2f}".format)
    growth_median_results['R-Squared(%)'] = growth_median_results['R-Squared(%)'].map("{:.2f}%".format)

    return growth_median_results

In [ ]:
# Get USD/CNH Exchange Rate
fx_data = w.edb("M0000185", "2009-01-01", end_date, Period="M")
# Parse the data
times = fx_data.Times
data = fx_data.Data[0]
times = pd.to_datetime([str(time) for time in times], format="%Y-%m-%d")

# Construct the DataFrame
df_forex = pd.DataFrame(data, index=times, columns=["USD/CNY Central Parity Rate"])
df_forex = logyoy(df_forex, df_forex.columns)

# Display the DataFrame
df_forex

## Growth Factor

In [ ]:
# consider key growth indicators for China
china_growth_inds = {
    # production method
    # general macro indicators
    'GDP_China(%)': 'M0039354',
    'Industrial_Growth(%)': 'M0000545',
    'PMI': 'M0017126',
    'PMI_manufacture': 'M0017127',
    'PMI_new_orders': 'M0017128',
    # by industry
    'electricity(%)': 'S0027013',
    'concrete(%)': 'S0027703',
    'steel(%)': 'S0027375',
    'automobile(%)': 'S0027908',
    'railroad_cargo(%)': 'S0036034',
    # expenditure method
    # investments
    'fixed_asset(%)': 'M0000273',
    'real_estate(%)': 'S0029657',
    'infrastructure(%)': 'M5440435',
    'manufacture(%)': 'M0000357',
    # consumption
    'retail(%)': 'M0001428',
    'automobile_sales(%)': 'S6114593',
    'tractor_sales(%)': 'S6002167',
    'commercial_RE_area(%)': 'S0073300',
    'commercial_RE_revenue(%)': 'S0049591',
    # net exports
    'import_export(%)': 'M0000605',
    'export(%)': 'M0000607',
    'import(%)': 'M0000609',
    'export_to_US(%)': 'M0008499',
    # income method
    'govt_revenue(%)': 'M0046169',
    'industrials_biz_income(%)': 'M0000555',
    'industrials_tot_profits(%)': 'M0000557'
}
# for standardizing the PMI data
china_nonyoy_cols = ['PMI', 'PMI_manufacture', 'PMI_new_orders']

In [ ]:
# actual step retrieving data from Wind API
df_china_growth = extract_inds(china_growth_inds)
df_china_growth = lag_data(df_china_growth)  # lag data by 1 period
df_china_growth = logyoy(df_china_growth, china_nonyoy_cols)
df_china_growth = df_china_growth.bfill().interpolate(method="linear")  # interpolate missing values
df_adjusted = apply_x11(df_china_growth, df_china_growth.columns[1:])
df_china_growth.head(10)

In [29]:
df_adjusted = apply_x11(df_china_growth, df_china_growth.columns[1:])
df_china_growth.head(10)

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/statsmodels/tsa/x13.py:192: X13Warning: WARNING: At least one visually significant seasonal peak has been found
          in the estimated spectrum of the regARIMA residuals.
  warn(errors, X13Warning)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/statsmodels/tsa/x13.py:192: X13Warning: WARNING: At least one visually significant seasonal peak has been found
          in the estimated spectrum of the regARIMA residuals.
  warn(errors, X13Warning)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/statsmodels/tsa/x13.py:192: X13Warning: WARNING: At least one visually significant seasonal peak has been found
          in the estimated spectrum of the regARIMA residuals.
  warn(errors, X13Warning)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/statsmodels/tsa/x13.py:192: X13Warning: WARNING: At least one visually sig

Error processing column industrials_biz_income(%): ERROR: Adding AO2014.Jan exceeds the number of regression effects allowed
        in the model (80).

        Check the regression model, change the automatic outlier options,
        (e.g. method to ADDONE, raise the critical value, or change types
        to identify AOs only), or change the program limits (see Section 2.7
        of the X-13ARIMA-SEATS Reference Manual).


,GDP_China(%),Industrial_Growth(%),PMI,PMI_manufacture,PMI_new_orders,electricity(%),concrete(%),steel(%),automobile(%),railroad_cargo(%),...,tractor_sales(%),commercial_RE_area(%),commercial_RE_revenue(%),import_export(%),export(%),import(%),export_to_US(%),govt_revenue(%),industrials_biz_income(%),industrials_tot_profits(%)
2010-01-31,11.9,18.5,31.757073,44.364402,49.188054,25.9000,12.600,26.6000,130.5000,18.4,...,136.73,42.1,75.5,32.76,17.73,55.94,-12.5,11.70,39.69,119.69
2010-02-28,12.2,29.2,20.846684,28.493104,28.601402,36.4625,49.024,28.3399,144.3037,18.0,...,251.07,38.2,70.2,44.53,21.00,85.96,8.4,41.20,39.69,119.69
2010-03-31,12.2,12.8,5.942342,5.878469,6.342183,7.9000,4.800,22.5000,46.7000,17.2,...,2.43,38.2,70.2,45.36,45.63,45.07,20.9,32.90,39.69,119.69
2010-04-30,12.2,18.1,5.024312,2.602055,6.213178,17.6000,12.100,22.5000,51.5000,17.4,...,126.32,35.8,57.7,42.86,24.21,66.27,19.7,34.00,38.17,81.64
2010-05-31,10.8,17.8,4.029849,2.918662,4.660032,21.4000,16.100,27.0000,34.6000,14.1,...,103.44,32.8,55.4,39.43,30.38,49.96,19.6,34.10,38.17,81.64
2010-06-30,10.8,16.5,1.495355,2.259001,-2.522656,18.9000,18.000,20.7000,26.6000,12.8,...,122.35,22.5,38.4,48.37,48.44,48.30,24.8,30.85,38.17,81.64
2010-07-31,10.8,13.7,-2.089345,-2.303025,-6.321807,11.4000,14.600,9.0000,18.4000,10.3,...,76.11,15.4,25.4,39.09,43.87,33.87,28.3,27.59,33.37,55.01
2010-08-31,9.9,13.4,-4.019680,-8.368517,-8.652010,11.5000,16.600,2.2000,17.1000,8.9,...,54.07,9.7,16.8,30.81,37.99,22.85,29.4,25.70,33.37,55.01
2010-09-30,9.9,13.9,-4.352627,-8.654046,-5.851761,12.6000,12.800,-1.1000,13.1000,7.0,...,41.24,6.7,12.6,34.81,34.32,35.40,31.2,23.60,33.37,55.01
2010-10-31,9.9,13.3,-0.925076,-2.797385,-0.884179,8.1000,10.300,-5.9000,17.8000,7.2,...,48.07,8.2,15.9,24.75,25.08,24.38,30.7,22.40,31.78,49.35


In [30]:
china_growth_ctrb = regress(df_china_growth, df_forex)
china_growth_ctrb

,Indicator,Beta,T-Value,R-Squared(%)
0,GDP_China(%),-0.71,-3.55,22.31%
1,Industrial_Growth(%),-0.29,-2.68,17.79%
2,PMI,-0.02,-0.13,1.99%
3,PMI_manufacture,0.04,0.29,2.58%
4,PMI_new_orders,0.00,0.01,2.17%
5,automobile(%),-0.02,-0.58,2.61%
6,automobile_sales(%),-0.02,-0.56,3.12%
7,commercial_RE_area(%),-0.07,-1.67,9.82%
8,commercial_RE_revenue(%),-0.05,-1.54,8.03%
9,concrete(%),-0.07,-1.01,4.51%


In [31]:
us_growth_inds = {
        # production method
        # general macro indicators
        'GDP_US': 'G1112986',
        'industrial production': 'G1109285',
        'PMI': 'G0002323',
        'PMI_output': 'G0008346',
        'PMI_new_order': 'G0008345', 
        # by industry
        'electricity': 'G1130815',
        'concrete_rebar': 'D5715886',
        'crude_steel': 'S5709965',
        'automobile': 'G1121656',
        # investment
        'real_estate_loans': 'T7212500',
        # consumption
        'new_house_sales': 'G0003650',
        'auto_sales': 'G1121638',
        'food_and_retail_sales': 'G1113053',
        'import_export': 'G1113168',
        'export': 'G1113166',
        'import': 'G1100355',
        'export_to_China': 'G1101044'
}
us_nonyoy_cols = ['PMI', 'PMI_output', 'PMI_new_order', 'electricity', 'concrete_rebar', 'automobile', 'real_estate_loans', 'new_house_sales', 'auto_sales', 'export_to_China']
us_needsa_cols = ['PMI', 'PMI_output', 'PMI_new_order', 'electricity', 'crude_steel', 'auto_sales', 'food_and_retail_sales', 'import_export', 'export', 'export_to_China']

In [33]:
df_us_growth = extract_inds(us_growth_inds)
df_us_growth = lag_data(df_us_growth)  # lag data by 1 period
df_us_growth = logyoy(df_us_growth, us_nonyoy_cols)
df_us_growth = df_us_growth.bfill().interpolate(method="linear")  # interpolate missing values
df_us_growth = apply_x11(df_us_growth, us_needsa_cols)
df_us_growth.head(10)

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/statsmodels/tsa/x13.py:192: X13Warning: WARNING: At least one visually significant trading day peak has been
          found in the estimated spectrum of the regARIMA residuals.
  
          found in one or more of the estimated spectra.
  warn(errors, X13Warning)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/statsmodels/tsa/x13.py:192: X13Warning: WARNING: At least one visually significant trading day peak has been
          found in the estimated spectrum of the regARIMA residuals.
  
          found in one or more of the estimated spectra.
  warn(errors, X13Warning)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/statsmodels/tsa/x13.py:192: X13Warning: WARNING: At least one visually significant trading day peak has been
          found in the estimated spectrum of the regARIMA residuals.
  
          found in one or more of the estimated s

,GDP_US,industrial production,PMI,PMI_output,PMI_new_order,electricity,concrete_rebar,crude_steel,automobile,real_estate_loans,new_house_sales,auto_sales,food_and_retail_sales,import_export,export,import,export_to_China
2010-01-31,0.1055,-2.63,51.287091,80.988054,101.382106,3.172503,-19.857059,47.519879,8.253568,-5.194266,-6.861401,20.453938,4.607825,-0.153071,14.095699,4.84,51.186213
2010-02-28,1.7450,0.92,48.886463,74.913490,67.034018,2.714527,-13.778206,49.888187,89.036216,-5.553888,2.643326,13.191839,2.502375,4.384006,18.543055,11.05,47.291520
2010-03-31,1.7450,1.89,44.183095,50.350207,51.668454,4.197589,-18.252293,53.258041,66.103261,-6.084394,-10.178269,14.114363,3.944363,-38.994388,19.336046,20.22,39.229581
2010-04-30,1.7450,4.24,50.253880,50.664425,43.208175,1.284912,-17.383215,77.357597,47.872515,-6.733902,11.679927,17.836818,9.627351,-40.596893,24.870394,22.14,32.709769
2010-05-31,2.9141,5.44,37.945906,45.584810,31.887244,-0.129475,-15.082623,79.869056,51.333798,-7.064038,22.492238,12.390446,7.424230,-32.196264,24.472931,22.92,24.789969
2010-06-30,2.9141,7.95,29.899838,44.684940,19.655852,6.027510,-21.463889,73.957758,66.781937,-7.659409,-29.479954,11.986305,4.655307,-44.822829,25.097588,27.89,25.675776
2010-07-31,2.9141,8.51,20.989875,14.613233,14.883326,7.387114,-23.306567,66.122501,58.060469,-7.906649,-25.349784,9.893419,4.690811,-58.730806,22.873007,27.84,19.105111
2010-08-31,3.3443,7.64,13.199663,-4.642212,-0.638910,7.998497,-23.272698,30.379043,41.227563,-8.171898,-37.314632,-1.280623,5.188123,-16.985820,21.913500,20.09,34.213611
2010-09-30,3.3443,6.84,9.229252,0.491703,-10.526977,6.983033,-22.910244,21.055524,25.202353,-8.314473,-39.357436,-36.893444,1.419275,-49.223707,21.093339,23.84,25.082115
2010-10-31,3.3443,6.24,4.447816,-1.113679,-14.178288,5.063626,-19.583034,15.462934,9.021936,-8.560331,-19.693560,14.918915,6.446714,-25.732834,19.007091,17.80,22.339352


In [34]:
us_growth_ctrb = regress(df_us_growth, df_forex)
us_growth_ctrb

,Indicator,Beta,T-Value,R-Squared(%)
0,GDP_US,-0.50,-1.46,5.57%
1,PMI,-0.15,-2.44,21.97%
2,PMI_new_order,-0.10,-2.71,20.44%
3,PMI_output,-0.09,-2.22,13.93%
4,auto_sales,-0.04,-1.01,3.83%
5,automobile,-0.01,-0.45,3.00%
6,concrete_rebar,0.17,2.85,17.83%
7,crude_steel,-0.06,-1.40,5.39%
8,electricity,-0.24,-1.07,4.50%
9,export,-0.13,-1.98,11.71%


It appears that of all the indicators picked intended to represent growth in China and US's economy, most of the indicators have very little contributions to the movement of USD/CNH. China's GDP appears to have a more direct impact, as its T-value indicates significance and has a relatively high R-squared value of roughly 20%, thus we select China's GDP as a component of our growth factor. Meanwhile, the US GDP appears less impactful than its counterpart but is still the most influential indicator amongst the US indicators. Hence, we also select the US GDP as a component of the growth factor. 

Both China and US GDP having slight negative beta on the exchange rate (USD/CNY) can be explained by considering the economic dynamics between the two countries. In China's case, a growing Chinese economy might lead to increased export which prompts RMB to appreciate against USD, while a growing US economy often leads to increased imports, which increases demands for Chinese goods and thereby exerts downward pressure on USD/CNH exchange rate.

In [ ]:
# combine the factors based on Inverse Volatility Weights
invvol_cn = 1 / df_china_growth["GDP_China(%)"].std()
invvol_us = 1 / df_us_growth["GDP_US"].std()
total_invvol = invvol_cn + invvol_us
weight_cn = invvol_cn / total_invvol
weight_us = invvol_us / total_invvol
growth = (weight_us * df_us_growth["GDP_US"]) + (weight_cn * df_china_growth["GDP_China(%)"])
growth


In [ ]:
# Calculate rolling average (smoothing)
window_size = 2  # You can adjust the window size for more or less smoothing
smoothed_us_gdp = df_us_growth["GDP_US"].rolling(window=window_size).mean()
smoothed_china_gdp = df_china_growth["GDP_China(%)"].rolling(window=window_size).mean()
smoothed_growth = growth.rolling(window=window_size).mean()

# Plotting the data
plt.figure(figsize=(10, 6))
plt.plot(smoothed_us_gdp.index, smoothed_us_gdp, label='US GDP (Smoothed)')
plt.plot(smoothed_china_gdp.index, smoothed_china_gdp, label='China GDP (Smoothed)')
plt.plot(smoothed_growth.index, smoothed_growth, label='Growth Factor (Smoothed)')
plt.xlabel('Year')
plt.ylabel('GDP(%)')
plt.title('Growth Factor Components')
plt.legend()
plt.grid(True)
plt.show()

## Inflation Factor